In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import tensorflow as tf
import sys

sys.path.append('../../')

# Data Input functions

In [2]:
from dnntc.datasets import CsvDataset

train_dataset = CsvDataset(
    filenames='../data/op_spam_v1.4/train_*.csv',
    vocab_file='../data/op_spam_v1.4/vocab.csv',
    batch_size=128
)
train_input_fn = train_dataset.input_fn


* data input_fn:
Input file(s): ../data/op_spam_v1.4/train_*.csv
Batch size: 128
Epoch Count: None
Thread Count: 8
Shuffle: True



# Estimator

In [3]:
from dnntc.models.cnn import model_fn

run_config = tf.estimator.RunConfig(
    log_step_count_steps=100,
    tf_random_seed=0
)

feature_columns = [
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_file(
            key='text', vocabulary_file='../data/op_spam_v1.4/vocab.csv'
        ),
        dimension=5
    )
]

hparams = tf.contrib.training.HParams(
    embedding_size = 5,
    window_size = 3,
    filters = 10,
    hidden_units=None, #[10],
    max_steps = 200,
    learning_rate = 0.001,
    feature_columns=feature_columns
)

estimator = tf.estimator.Estimator(
    model_fn=model_fn, 
    params=hparams, 
    config=run_config
)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hh/lmg2ypzn7fz3hbg8h2vw4n5h0000gn/T/tmphi0lz6rq', '_tf_random_seed': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121741ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train

In [4]:
# DEBUG, INFO, WARN, ERROR, FATAL
tf.logging.set_verbosity(tf.logging.INFO)
result = pd.DataFrame()

In [5]:
from IPython.display import clear_output

for _ in range(3):
    estimator.train(input_fn=train_input_fn, steps=100)
    res = estimator.evaluate(input_fn=valid_input_fn)

    result = result.append(pd.DataFrame([res]))

    clear_output(wait=True)
    result['F1'] = 2 * (
        (result['precision'] * result['recall']) / (result['precision'] + result['recall']))

    display(result[['global_step', 'loss', 'accuracy', 'precision', 'recall', 'F1']].set_index('global_step'))

INFO:tensorflow:Calling model_fn.


ValueError: Tensor("conv1d/kernel:0", shape=(3, 5, 10), dtype=float32_ref) must be from the same graph as Tensor("EmbedSequence/embedding_lookup:0", shape=(?, 400, 5), dtype=float32).

# Predict

In [18]:
test_data = pd.read_hdf('../data/trajectories.h5', key='data', start=0, stop=128)
print(len(test_data))
test_data.head(2)

128


,connector_guid,date,detection,detection_id,endpoint_ioc_scan.clean,endpoint_ioc_scan.description,endpoint_ioc_scan.matched_objects,endpoint_ioc_scan.scanned_objects,error.description,error.error_code,...,scan.description,scan.malicious_detections,scan.scanned_files,scan.scanned_paths,scan.scanned_processes,start_date,start_timestamp,timestamp,timestamp_nanoseconds,vulnerabilities
0,297aa824-9afd-484a-b7e3-b9a8b4f8cc16,2017-11-28T15:36:04+00:00,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,...,Flash Scan,0.0,274.0,0.0,4.0,NaN,NaN,1511883364,356000000,NaN
1,297aa824-9afd-484a-b7e3-b9a8b4f8cc16,2017-11-28T15:35:53+00:00,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,...,Flash Scan,NaN,NaN,NaN,NaN,NaN,NaN,1511883353,159000000,NaN


In [22]:
from cisco.datasets import CustomDataset

dataset = CustomDataset(data=test_data, batch_size=128)
input_fn = dataset.input_fn

predictions = pd.DataFrame(list(estimator.predict(input_fn=input_fn)))
predictions

INFO:tensorflow:Initializing Dataset
INFO:tensorflow:Data n: 128, Have labels = True
INFO:tensorflow:Calling model_fn.
None
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/hh/lmg2ypzn7fz3hbg8h2vw4n5h0000gn/T/tmpoqrs_4ay/model.ckpt-201
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


,prediction,sigmoids
0,1.0,0.615108
1,1.0,0.579512
2,1.0,0.615108
3,0.0,0.487290
4,1.0,0.615108
5,0.0,0.487290
6,1.0,0.747282
7,1.0,0.621345
8,1.0,0.615108
9,1.0,0.615108


# Plot

In [ ]:
result.set_index('global_step')[['accuracy', 'precision', 'recall', 'F1']].plot()